In [1]:
pip install pygame


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pygame
import sys
import random

# Initialize pygame
pygame.init()

# Screen setup
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Realistic Bus Parking Simulator")

# Colors
WHITE = (255, 255, 255)
GRAY = (169, 169, 169)  # Road color
GREEN = (0, 255, 0)     # Parking zone color
RED = (255, 0, 0)       # Obstacle color
BLUE = (0, 0, 255)      # Bus outline
BLACK = (0, 0, 0)

# Clock
clock = pygame.time.Clock()

# Load bus image
bus_width, bus_height = 70, 150
bus = pygame.image.load("bus-png-4.png")  # Place the bus.png file in the same directory
bus = pygame.transform.scale(bus, (bus_width, bus_height))

# Obstacle dimensions
obstacle_width, obstacle_height = 50, 50
obstacle_speed = 2

# Font
font = pygame.font.SysFont(None, 36)


def draw_text(text, color, x, y):
    """Helper function to draw text on the screen."""
    message = font.render(text, True, color)
    screen.blit(message, (x, y))


def bus_parking_game():
    # Bus starting position
    bus_x, bus_y = WIDTH // 2 - bus_width // 2, HEIGHT - bus_height - 10
    bus_speed = 5

    # Parking zone dimensions
    parking_width, parking_height = 150, 100
    parking_x, parking_y = WIDTH // 2 - parking_width // 2, 50

    # Generate random obstacles
    obstacles = [
        {"x": random.randint(100, WIDTH - 100), "y": random.randint(200, HEIGHT - 300), "direction": random.choice([-1, 1])}
        for _ in range(3)
    ]

    running = True
    while running:
        screen.fill(WHITE)

        # Event handling
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        # Movement
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT] and bus_x > 0:
            bus_x -= bus_speed
        if keys[pygame.K_RIGHT] and bus_x < WIDTH - bus_width:
            bus_x += bus_speed
        if keys[pygame.K_UP] and bus_y > 0:
            bus_y -= bus_speed
        if keys[pygame.K_DOWN] and bus_y < HEIGHT - bus_height:
            bus_y += bus_speed

        # Draw road (Realistic)
        pygame.draw.rect(screen, GRAY, (50, 0, WIDTH - 100, HEIGHT))  # Main road
        pygame.draw.rect(screen, GRAY, (WIDTH // 3, HEIGHT - 200, WIDTH // 3, 200))  # Final parking lane

        # Draw parking zone
        pygame.draw.rect(screen, GREEN, (parking_x, parking_y, parking_width, parking_height))

        # Visual parking guide
        if parking_x <= bus_x <= parking_x + parking_width - bus_width and parking_y <= bus_y <= parking_y + parking_height:
            draw_text("Align Properly!", BLUE, 300, 10)
        else:
            draw_text("Not Parked Correctly!", RED, 250, 10)

        # Draw obstacles (Static and moving cars)
        for obstacle in obstacles:
            pygame.draw.rect(screen, RED, (obstacle["x"], obstacle["y"], obstacle_width, obstacle_height))
            obstacle["y"] += obstacle_speed * obstacle["direction"]

            # Bounce obstacle direction
            if obstacle["y"] <= 0 or obstacle["y"] >= HEIGHT - obstacle_height:
                obstacle["direction"] *= -1

            # Collision detection
            if (
                bus_x < obstacle["x"] + obstacle_width and
                bus_x + bus_width > obstacle["x"] and
                bus_y < obstacle["y"] + obstacle_height and
                bus_y + bus_height > obstacle["y"]
            ):
                draw_text("Crashed! Restarting...", RED, 250, HEIGHT // 2)
                pygame.display.update()
                pygame.time.wait(2000)
                return  # Restart the game

        # Draw bus
        screen.blit(bus, (bus_x, bus_y))

        # Check if the bus is fully parked
        if (
            parking_x <= bus_x <= parking_x + parking_width - bus_width and
            parking_y <= bus_y <= parking_y + parking_height
        ):
            draw_text("Parked Successfully! Restarting...", BLACK, 200, HEIGHT // 2)
            pygame.display.update()
            pygame.time.wait(2000)
            return  # Restart the game

        pygame.display.update()
        clock.tick(60)


# Main loop
while True:
    bus_parking_game()
